In [4]:
import pandas as pd
import csv

In [16]:
csv_path = 'Full Text Info.csv'

# 定義自定義解析器
def custom_csv_parser(file_path, encoding='ISO-8859-1'):
    rows = []
    with open(file_path, encoding=encoding) as file:
        reader = csv.reader(file)
        for row in reader:
            if len(row) > 8:
                row[7] = ', '.join(row[7:])
                row = row[:8]
            print(row)
            row[-1] = row[-1].rstrip(', ')
            rows.append(row)
    return rows

# 使用自定義解析器讀取文件
rows = custom_csv_parser(csv_path)

# 定義標題列
columns = ["FileName", "EntityName", "ObjectType", "RotationAngle", "CentreCoor", "Height", "Width", "Text"]

# 將結果轉換為 DataFrame
df = pd.DataFrame(rows[1:], columns=columns)

['FileName', 'EntityName', 'ObjectType', 'RotationAngle', 'CentreCoor', 'Height', 'Width', 'Text, ']
['Q881LG10SE1470.dwg', '<¹Ï¤¸¦WºÙ: 16f60a8c200>', 'AcDbText', '0.0', '(312534.36 2623473.53 0)', '0.26', '1.0', 'SE 021, ']
['Q881LG10SE1470.dwg', '<¹Ï¤¸¦WºÙ: 16f60a8c1f0>', 'AcDbText', '0.0', '(312534.36 2623472.13 0)', '0.26', '1.0', 'LAP  SPLICES  SHOULD  BE  THAT  OF  DRG  SE 021, ']
['Q881LG10SE1470.dwg', '<¹Ï¤¸¦WºÙ: 16f60a8c1e0>', 'AcDbText', '0.0', '(312534.36 2623471.33 0)', '0.26', '1.0', "fc'=245 kgf/cm\\U+00B2   OF  DEFORMED  BARS ., "]
['Q881LG10SE1470.dwg', '<¹Ï¤¸¦WºÙ: 16f60a8c1d0>', 'AcDbText', '0.0', '(312539.48 2623473.53 0)', '0.26', '1.0', "fc'=245 kgf/cm\\U+00B2, "]
['Q881LG10SE1470.dwg', '<¹Ï¤¸¦WºÙ: 16f60a8c1c0>', 'AcDbText', '0.0', '(312534.36 2623472.99 0)', '0.35', '0.85', '¦A\xad¼¥H        \xad¿±o¤§¡C, ']
['Q881LG10SE1470.dwg', '<¹Ï¤¸¦WºÙ: 16f60a8c1b0>', 'AcDbText', '0.0', '(312534.36 2623473.99 0)', '0.35', '0.85', '¦³Ãö³sÄò¾À¿ûµ¬¤§®I¤Jªø«×¤Î·f±µªø«×¤§³W©w½Ð¨Ì¹Ï

In [22]:
rows, columns = df.shape
for i in range(rows):
    print(df.iloc[i]['Text'])

SE 021
LAP  SPLICES  SHOULD  BE  THAT  OF  DRG  SE 021
fc'=245 kgf/cm\U+00B2   OF  DEFORMED  BARS .
fc'=245 kgf/cm\U+00B2
¦A­¼¥H        ­¿±o¤§¡C
¦³Ãö³sÄò¾À¿ûµ¬¤§®I¤Jªø«×¤Î·f±µªø«×¤§³W©w½Ð¨Ì¹Ï
ªí    ¤¤¦Ë¸`¿ûµ¬²V¾®¤g                           ¤§ª¬ªp¡A
TABLE  2  MULTIPLIED  BY  1.33  ACCORDING  TO
THE  EMBEDMENT  LENGTH  AND  THE  LENGTH  OF
1.33
2
6.
(                       )
ON  PRIMARY PANEL  OF DIAPHRAGM  WALL
AROUND  WORKING  SHAFT  AREA .
U-SHAPE  END  PLATE  SHOULD  BE  INSTALLED
U
¤§ºÝªO¡C
 ¤½³æ¤¸°Ï°ì ¨â­±»Ý¥[ªø¡A§Y¥~Æ[§e      ¦r«¬
¤u§@¤«°Ï°ì³sÄò¾À¥À³æ¤¸¿û½èºÝªO©ó¶W«õ¬q
10.
¶}«õ°¼
(               )
(EXCAVATED  SIDE)
¾F±µ³æ¤¸¿ûµ¬¥¼¥Ü
NOT  SHOWN
REINFORCEMENT
ADJACENT  PANEL
¾×¤g°¼
(               )
(RETAINED  SIDE)
³sÄò¾À³æ¤¸¼e«×
PANEL  WIDTH
¿û½èºÝªO
(TYP)
STEEL  END  PLATE  (TYP)
NOTE  8
±×©Ôµ¬¸Ô¨£»¡©ú
DIAGONAL  BAR  SEE
8
NOTE  8
±×©Ôµ¬¸Ô¨£»¡©ú
DIAGONAL  BAR  SEE
8
1:100
¤ä¼µ¦ì¸m¹Ï
STRUT  LOCATION
A
B
C
(EXCAVATED  SIDE)
(            )
¶}«õ°¼
(RETAINED  SIDE)
(            )
¾×¤g

In [39]:
grouped = df.groupby('FileName')

# 遍歷每個組
for name, group in grouped:
    print(f"組名: {name}")
    
    # 查找包含 'DIAPHRAGM WALL TYPE' 的行
    wall_type_row = group[group['Text'].str.contains('DIAPHRAGM  WALL  TYPE', case=False)]
    
    # 查找包含 'EL' 後跟數字的行
    # 定義正則表達式模式來匹配 'EL' 後跟浮點數
    pattern = r'EL (\d+(?:\.\d+)?)'

    # 使用 str.extractall 提取所有匹配的值
    el_values = group['Text'].str.extractall(pattern)

    print("Extracted EL values before conversion:")
    print(el_values)

    # 重置索引，將 match 級別的索引轉為列
    el_values = el_values.reset_index(level='match', drop=True)

    print("Extracted EL values after reset_index:")
    print(el_values)

    # 重命名列
    el_values.columns = ['EL_Value']

    # 將提取的值轉換為浮點數
    el_values['EL_Value'] = el_values['EL_Value'].astype(float)

    # show the extracted EL values
    # print("Extracted EL values:")
    # print(el_values)
    
    if wall_type_row.empty:
        print("No row contains 'Diaphragm Wall Type'")
    else:
        print("Diaphragm Wall Type:")
        print(wall_type_row['Text'].values[0] if len(wall_type_row) > 0 else "Multiple matches found")
    
    print("\n" + "="*50 + "\n")

組名: Q881LG10SE1470.dwg
Extracted EL values before conversion:
                0
    match        
91  0      108.85
95  0      104.75
96  0       97.72
121 0      110.70
135 0       83.35
180 0       83.35
192 0       83.35
202 0       90.85
203 0      107.85
204 0       90.85
205 0      100.85
208 0      103.85
Extracted EL values after reset_index:
          0
91   108.85
95   104.75
96    97.72
121  110.70
135   83.35
180   83.35
192   83.35
202   90.85
203  107.85
204   90.85
205  100.85
208  103.85
Diaphragm Wall Type:
DIAPHRAGM  WALL  TYPE  T1


組名: Q881LG10SE1480.dwg
Extracted EL values before conversion:
               0
    match       
356 0      96.05
434 0      83.35
444 0      83.35
451 0      90.85
452 0      90.85
455 0      96.05
Extracted EL values after reset_index:
         0
356  96.05
434  83.35
444  83.35
451  90.85
452  90.85
455  96.05
Diaphragm Wall Type:
DIAPHRAGM  WALL  TYPE  T1A


組名: Q881LG10SE1490.dwg
Extracted EL values before conversion:
                